In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc
from lightgbm import LGBMClassifier



SAMPLE_SIZE = 0.5
%matplotlib inline

In [2]:
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
import random
import gc
class RandomLightGBM:

    
    def __init__(self,  n_iterations, n_features, retries, subset_sample, verbose, meta_parameters = []):
        self.n_iterations=n_iterations
        self.n_features=n_features
        self.retries=retries
        self.subset_sample=subset_sample
        self.verbose=verbose
        self.LGBMs = []
        self.chosen_features=[]
        self.meta_parameters = meta_parameters
    
    
    def roc_auc_score(self, test, to_predict):
        y_test=test[to_predict]
        return roc_auc_score(y_test, self.predict_proba_class(1,test))
    
    def predict_proba_class(self,number,data):
        n_models = len(self.LGBMs)
        if(n_models == 0):
            raise Exception('No naive bayes has been trained.')
        preds=[]
        for i in range(0,n_models):
            y=self.LGBMs[i].predict_proba(data[self.chosen_features[i]])
            y_pred=[]
            for elem in y:
                y_pred.append(elem[number])
            preds.append(y_pred)
        y_pred=[sum(x) for x in zip(*preds)]
        y_pred=[i / n_models for i in y_pred]
        return y_pred
    
    def score_iteration(self,train,features,to_predict):
        #TODO: Configurar para pasarle metaparametros
        c=LGBMClassifier(learning_rate=0.005,objective='binary',num_leaves=300,max_depth=9,n_estimators=1000,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)
        subsample=train.sample(int(round(len(train)*self.subset_sample)))
        c.fit(subsample[features],subsample[to_predict])
        self.LGBMs.append(c)
        self.chosen_features.append(features)
        return self.roc_auc_score(test,to_predict)
        
    def fit(self,train, test, to_predict, features):
        best=0
        
        #Por cada iteracion, entrena un clasificador LGBM con los meta parametros, un subsample aleatorio del set de entrenamiento
        #de tamano subsample y compara los scores
        for i in range(1,self.n_iterations+1):
            features_aux=[]
            if(i==1):
                for j in range(0,self.n_features):
                    features_aux.append(random.choice(features))
            else:
                features_aux += 
            if(self.verbose):
                print("Iteration "+str(i)+" with features "+str(features_aux))
            score = self.score_iteration(train,features_aux,to_predict)
            if(self.verbose):
                print("Score: "+str(score)+" with features "+str(features_aux))
           
            if(score<=best):
                for k in range(0,self.retries):
                    features_aux=[]
                    self.LGBMs.pop()
                    self.chosen_features.pop()
                    for j in range(0,self.n_features):
                        features_aux.append(random.choice(features))
                    score = self.score_iteration(train,features_aux,to_predict)
                    if(self.verbose):
                        print("Retry number "+str(k+1)+" with score: "+str(score)+" and features: "+str(features_aux))
                    if(score>best):
                        break
            if(score<=best):
                self.LGBMs.pop()
                self.chosen_features.pop()
                print("Stopping")
                return;
            best=score
            subsample=0
            gc.collect()

In [3]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisos.csv")
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,1
1,48375,RwVdKR,1
2,169730,1KjXB,1
3,169730,2AKzxa,1
4,169730,6LJ64,1


In [5]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

,idpostulante,cantidad
0,0002q,3
1,0005E,25
2,000R8,4
3,001XE,12
4,003k9,26


In [6]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,1,20
1,48375,RwVdKR,1,10
2,169730,1KjXB,1,34
3,169730,2AKzxa,1,1
4,169730,6LJ64,1,10


In [7]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.050000
1,48375,RwVdKR,0.100000
2,169730,1KjXB,0.029412
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.100000


In [8]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})
vistas_final = pd.merge(vistas3,avisos, on= "idaviso", how = "left")
vistas_final.head()

,idaviso,idpostulante,cantidad,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,titulo_uppercase,...,graduados,estudiantes,conocimientos_especificos,buen_ambiente,indica_sueldo,part_time,full_time,ofrece_crecimiento,ofrece_beneficios,obra_social
0,18,BolNL,0.050000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,48375,RwVdKR,0.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,169730,1KjXB,0.029412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,169730,2AKzxa,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,169730,6LJ64,0.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
no_postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv")
postulaciones=postulaciones.append(no_postulaciones, ignore_index=True)
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
postulaciones = postulaciones.sample(int(round(len(postulaciones)*SAMPLE_SIZE)))
no_postulaciones = 0
gc.collect()

39

In [10]:
vistas_final["vistas_por_area"]=0
vistas_final["vistas_por_tipo"]=0
vistas_final["vistas_por_nivel"]=0

vistas_area = vistas_final.groupby(['idpostulante','nombre_area'],as_index=False).agg({'vistas_por_area':'count'})
vistas_tipo= vistas_final.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_nivel=vistas_final.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})

vistas_final = 0
gc.collect()

21

In [11]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones["vistas_al_aviso"] = postulaciones["vistas_al_aviso"].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)


postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,ofrece_crecimiento,ofrece_beneficios,obra_social,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio
1503658,1112379944,BmDaaov,1,MASC,25.0,-,-,-,-,Graduado,...,False,False,False,22.0,0.0,NaN,1.0,22.0,22.0,False
1081350,1112460278,OqP5EXv,1,MASC,36.0,-,-,-,-,Graduado,...,False,False,False,21.0,0.0,NaN,13.0,19.0,19.0,False
4988767,1112325229,Yj5eXRP,0,MASC,20.0,-,-,-,-,-,...,False,False,False,2.0,0.0,NaN,0.0,1.0,1.0,False
548592,1112312666,5mP3GLZ,1,FEM,25.0,-,-,-,-,Graduado,...,False,False,False,69.0,0.0,NaN,11.0,46.0,46.0,False
2792652,1112288998,akje1e5,0,MASC,33.0,-,-,-,-,-,...,False,False,False,16.0,0.0,NaN,0.0,10.0,10.0,False
1378987,1112402895,aNlGxN,1,FEM,28.0,-,-,-,-,Graduado,...,False,False,False,45.0,0.0,NaN,2.0,30.0,30.0,False
378883,1112214523,BmxJKMa,1,MASC,26.0,-,-,-,-,Graduado,...,False,False,False,NaN,0.0,NaN,0.0,0.0,0.0,False
5441974,1111444739,JBrD8Rr,0,MASC,26.0,-,-,-,-,Graduado,...,False,False,True,25.0,0.0,NaN,2.0,19.0,19.0,False
3765540,1112245499,akx4w5q,1,FEM,25.0,-,-,-,-,Graduado,...,False,True,False,NaN,0.0,NaN,0.0,0.0,0.0,False
1290104,1112351932,EkN5kJ,1,MASC,31.0,-,-,-,-,Graduado,...,False,False,False,6.0,1.0,0.166667,1.0,6.0,6.0,True


In [12]:
postulaciones_area = postulaciones.groupby(['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_tipo= postulaciones.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_nivel =postulaciones.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_area=postulaciones_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_tipo=postulaciones_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_nivel=postulaciones_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})

In [13]:
postulaciones.head()



,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,ofrece_crecimiento,ofrece_beneficios,obra_social,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio
0,1112293412,pzM6zd3,1,FEM,25.0,-,-,Graduado,-,Graduado,...,False,False,False,NaN,0.0,NaN,0.0,0.0,0.0,False
1,1112293412,JBrBrBW,1,FEM,30.0,-,-,-,-,Graduado,...,False,False,False,408.0,0.0,NaN,2.0,345.0,345.0,False
2,1112293412,LNPJkRp,1,FEM,27.0,-,-,-,-,Graduado,...,False,False,False,NaN,0.0,NaN,0.0,0.0,0.0,False
3,1112293412,5Mwjak,1,MASC,38.0,-,-,-,-,Graduado,...,False,False,False,692.0,0.0,NaN,7.0,596.0,596.0,False
4,1112293412,1P5zQN,1,FEM,33.0,-,-,-,En Curso,-,...,False,False,False,10.0,0.0,NaN,1.0,8.0,8.0,False


In [14]:
postulaciones=pd.merge(postulaciones,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)


In [15]:
postulaciones.sample(10)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
831801,1112280937,0zdKP0Y,1,FEM,22.0,-,-,-,-,Graduado,...,NaN,0.0,NaN,0.0,0.0,0.0,False,0,4,4
2879691,1112353594,59GvpX,0,FEM,28.0,-,-,-,-,Graduado,...,49.0,0.0,NaN,1.0,34.0,34.0,False,1,33,26
5919613,1112513266,5NbNQX,0,FEM,30.0,-,-,-,-,Graduado,...,1.0,0.0,NaN,0.0,1.0,1.0,False,2,4,2
5157383,1112496620,Yj5e9rV,0,FEM,21.0,-,-,-,-,-,...,2.0,0.0,NaN,0.0,2.0,2.0,False,0,9,2
4106629,1112451640,kPjRZeZ,0,FEM,46.0,-,-,-,-,Graduado,...,9.0,0.0,NaN,0.0,9.0,9.0,False,0,16,10
4935735,1112460415,xkaV6Px,0,FEM,24.0,-,-,-,-,Graduado,...,2.0,0.0,NaN,0.0,1.0,1.0,False,0,6,4
3575594,1112299038,8v0JNz,1,FEM,29.0,-,-,-,-,-,...,11.0,0.0,NaN,2.0,0.0,0.0,False,0,2,0
6317856,1112502948,KBeBblj,0,FEM,23.0,-,-,-,-,Graduado,...,13.0,0.0,NaN,0.0,5.0,5.0,False,0,1,2
6320718,1000537816,kPLvLxm,0,MASC,33.0,-,-,-,-,Graduado,...,1.0,0.0,NaN,0.0,1.0,1.0,False,8,25,17
5560128,1112205943,ow2V5pZ,0,FEM,30.0,-,-,-,-,Graduado,...,1.0,0.0,NaN,0.0,1.0,1.0,False,0,7,3


In [16]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

postulaciones=0

gc.collect()

Train:  5754527 Test:  639392


35

In [18]:
model = RandomLightGBM(n_iterations=10,n_features=10,retries=3,subset_sample=0.3,verbose=True)
dt = model.fit(train,test,'sepostulo',features)

Iteration 1
Score: 0.9614441046627982 with features ['sexo', 'edad', 'Doctorado', 'Master', 'Otro', 'Posgrado', 'Secundario', 'Terciario/Técnico', 'Universitario', 'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area', 'titulo_uppercase', 'pide_hombre', 'pide_mujer', 'ingles', 'experiencia', 'paquete_office', 'pide_excel', 'liderazgo', 'secundario', 'tecnico', 'viajar', 'empresa_importante', 'capacitacion', 'remuneracion_pretendida', 'graduados', 'estudiantes', 'conocimientos_especificos', 'buen_ambiente', 'indica_sueldo', 'part_time', 'full_time', 'ofrece_crecimiento', 'ofrece_beneficios', 'obra_social', 'vistas_postulante', 'vistas_al_aviso', 'cantidad', 'vistas_por_area', 'vistas_por_tipo', 'vistas_por_nivel', 'lo_vio', 'postulaciones_misma_area', 'postulaciones_mismo_tipo', 'postulaciones_mismo_nivel']
Iteration 2
Score: 0.953601219275614 with features ['capacitacion', 'tipo_de_trabajo', 'empresa_importante', 'indica_sueldo', 'conocimientos_especificos', 'tecnico', 'sexo

In [19]:
score=roc_auc_score(y_test, model.predict_proba_class(1,test))
print(score)


0.9614441046627982


In [20]:
joblib.dump(model, "Data/fiuba_entrenamiento/pocho/modelos/RandomLGBM2.pkl")

In [24]:
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.050000
1,48375,RwVdKR,0.100000
2,169730,1KjXB,0.029412
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.100000


In [28]:
prediccion = pd.read_csv("Data/fiuba_entrenamiento/test_final_100k.csv")
prediccion = pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
prediccion = pd.merge(prediccion,avisos,on='idaviso',how='inner')
prediccion = pd.merge(prediccion,vistas,on='idpostulante',how='left')
prediccion.sample(10)


prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion['cantidad']=prediccion['cantidad'].fillna(0)
prediccion['vistas_postulante']=prediccion['vistas_postulante'].fillna(0)
prediccion['vistas_al_aviso']=prediccion['vistas_al_aviso'].fillna(0)
prediccion['lo_vio']=(prediccion['vistas_al_aviso']>0)

prediccion=pd.merge(prediccion,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion=pd.merge(prediccion,vistas_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0)
prediccion.sample(10)

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
59907,66488,1112459907,ED2qL4,MASC,45.0,-,-,-,-,-,...,22.0,0.0,0.000000,False,0.0,5.0,4.0,0.0,0.0,0.0
75483,37469,1112413438,aGVMKq,MASC,32.0,-,-,-,-,Graduado,...,9.0,0.0,0.000000,False,0.0,10.0,5.0,0.0,7.0,7.0
51711,36771,1112412413,mzbd0JG,MASC,24.0,-,-,-,-,Abandonado,...,0.0,0.0,0.000000,False,0.0,4.0,4.0,0.0,0.0,0.0
7059,25660,1112334791,vVkzEZG,MASC,24.0,-,-,-,-,Graduado,...,37.0,1.0,0.027027,True,0.0,14.0,11.0,3.0,28.0,28.0
70514,15992,1112156749,exxv44,MASC,45.0,-,-,-,-,-,...,76.0,0.0,0.000000,False,1.0,5.0,5.0,0.0,57.0,57.0
94787,94136,1112483241,a0qLKW,MASC,31.0,-,-,-,-,Graduado,...,6.0,0.0,0.000000,False,1.0,10.0,9.0,0.0,5.0,5.0
79959,27419,1112350288,8PpKo3,MASC,44.0,-,-,-,-,Graduado,...,1.0,0.0,0.000000,False,0.0,7.0,0.0,0.0,1.0,1.0
21032,80530,1112464970,owjkdZr,FEM,37.0,-,-,-,-,Graduado,...,0.0,0.0,0.000000,False,0.0,0.0,4.0,0.0,0.0,0.0
54071,72426,1112463111,xk9v4xj,MASC,27.0,-,-,-,-,Graduado,...,0.0,0.0,0.000000,False,0.0,5.0,4.0,0.0,0.0,0.0
25266,40561,1112419784,kPjrMjj,MASC,18.0,-,-,Graduado,-,Graduado,...,160.0,1.0,0.006250,True,6.0,21.0,16.0,17.0,80.0,80.0


In [30]:
y_final=model.predict_proba_class(1,prediccion)
prediccion['sepostulo']=y_final
prediccion.head()


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,0.0,0.0,False,0.0,3.0,1.0,3.0,20.0,20.0,0.069324
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,0.0,0.0,False,2.0,29.0,0.0,0.0,3.0,3.0,0.342740
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,0.0,0.0,False,0.0,4.0,0.0,1.0,6.0,6.0,0.074239
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,0.0,0.0,False,0.0,4.0,0.0,0.0,0.0,0.0,0.018368
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,0.0,0.0,False,1.0,8.0,1.0,0.0,1.0,1.0,0.031388


In [31]:
prediccion=prediccion[['id','sepostulo']]
print(prediccion.sepostulo.mean())
prediccion.head()

0.509041500692947


,id,sepostulo
0,0,0.069324
1,1,0.342740
2,2,0.074239
3,3,0.018368
4,4,0.031388


In [32]:
prediccion.to_csv("Data/fiuba_entrenamiento/pocho/predicciones/pred-rlgbm2.csv",index=False)